In [68]:
from scipy import linalg
import time
import numpy as np

In [19]:
def get_banded(matrix: np.ndarray, lbw: int, upw: int) -> np.ndarray:
    rows, cols = matrix.shape
    total_bands = lbw + upw + 1

    # Initialize bands array with zeros of the required size
    bands = np.zeros((total_bands, cols), dtype=matrix.dtype)
    lengths = np.zeros(total_bands, dtype=int)

    # Extract the bands from the matrix
    for i in range(rows):
        for j in range(max(0, i - lbw), min(cols, i + upw + 1)):
            band_index = i - j + upw
            bands[band_index, lengths[band_index]] = matrix[i, j]
            lengths[band_index] += 1

    # Adjust alignment
    for idx in range(total_bands):
        current_length = lengths[idx]
        if idx < upw:  # Right-align upper bands
            bands[idx] = np.roll(bands[idx], cols - current_length)
        else:  # Left-align middle and lower bands (already left-aligned)
            bands[idx, current_length:] = 0  # Fill remaining cells with zeros

    return bands

# Band Matrix Solver

A **Band Matrix** is a sparse matrix whose non-zero entries are confined to a diagonal band, comprising the main diagonal and a few diagonals on either side. Band matrices appear frequently in numerical analysis, especially when dealing with differential equations and systems with local interactions.


### `linalg.solve_banded`
The `linalg.solve_banded` function in SciPy provides an efficient way to solve banded systems of linear equations. It leverages the fact that the majority of the entries are zeros, improving both computational efficiency and memory usage.

#### LU Decomposition
The solver applies LU decomposition, which is modified to handle banded matrices by storing only the relevant diagonals. This approach significantly reduces the computational complexity compared to working with a full matrix.

#### Tridiagonal Solver
A special case of the band matrix solver is the **tridiagonal solver**, where the lower and upper bandwidths are both equal to 1 (`lbw = 1, ubw = 1`). This specific case allows for even more efficient algorithms, such as the Thomas algorithm, which operates in linear time.

In [27]:
b = np.ones((5, ), dtype = np.float64)

A1 = np.array([
    [1, 2, 0, 0, 0],
    [1, 4, 1, 0, 0],
    [5, 0, 1, 2, 0],
    [0, 1, 2, 2, 1],
    [0, 0, 2, 1, 1]
], dtype = np.float64)

A2 = np.array([
    [2, 1, 0, 0, 0],
    [1, 2, 1, 0, 0],
    [0, 1, 2, 1, 0],
    [0, 0, 1, 2, 1],
    [0, 0, 0, 1, 2]
], dtype = np.float64)

In [20]:
A1_band = get_banded(A1, 2, 1)

In [ ]:
A2_band = get_banded(A2, 1, 1)

In [24]:
print(A1_band)

[[0. 2. 1. 2. 1.]
 [1. 4. 1. 2. 1.]
 [1. 0. 2. 1. 0.]
 [5. 1. 2. 0. 0.]]


In [25]:
print(A2_band)

[[0. 1. 1. 1. 1.]
 [2. 2. 2. 2. 2.]
 [1. 1. 1. 1. 0.]]


In [28]:
x1 = linalg.solve_banded((2, 1), A1_band, b)
x2 = linalg.solve_banded((1, 1), A2_band, b)

In [29]:
print(x1)

[ 0.42857143  0.28571429 -0.57142857 -0.28571429  2.42857143]


In [30]:
print(x2)

[0.5 0.  0.5 0.  0.5]




### Positive Definite Band Matrix Solver
A matrix is **Positive Definite** if it is symmetric (for real matrices) or Hermitian (for complex matrices) and all of its eigenvalues are positive. When the band matrix is also positive definite, additional optimizations can be applied during the solving process.

- **Symmetric or Hermitian Property:** Positive definite matrices are either symmetric (real-valued) or Hermitian (complex-valued). This property can be exploited to reduce computational costs when solving banded systems.
- **Cholesky Decomposition:** When the matrix is positive definite, a Cholesky decomposition can be applied, further improving efficiency over standard LU decomposition.
- **LDLᵀ Decomposition:** Another efficient technique for positive definite matrices is the **LDLᵀ decomposition**, where the matrix \( A \) is decomposed into \( A = LDL^T \). Here, \( L \) is a lower triangular matrix with unit diagonal entries, and \( D \) is a diagonal matrix. This decomposition is particularly useful when the matrix is symmetric and not necessarily positive definite, but becomes especially efficient when \( A \) is positive definite.

In [46]:
def get_band_h(matrix, lbw, upw):
    banded = get_banded(matrix, lbw, upw)

    n_row, n_col = banded.shape
    mid = (n_row // 2) + 1

    return banded[0:mid]

In [59]:
b1 = np.ones((4, ), dtype = np.float64)
b2 = np.ones((5, ), dtype = np.float64)

A1 = np.array([
    [8, 2-1j, 0, 0],
    [2 + 1j, 5, 1j, 0],
    [0, -1j, 9, -2 - 1j],
    [0, 0, -2 + 1j, 6]
], dtype = np.complex128)

A2 = np.array([
    [2, 1, 0, 0, 0],
    [1, 2, 1, 0, 0],
    [0, 1, 2, 1, 0],
    [0, 0, 1, 2, 1],
    [0, 0, 0, 1, 2]
], dtype = np.float64)

In [60]:
A1_band_h = get_band_h(A1, 1, 1)
A2_band_h = get_band_h(A2, 1, 1)

In [61]:
print(A1_band_h)

[[ 0.+0.j  2.-1.j  0.+1.j -2.-1.j]
 [ 8.+0.j  5.+0.j  9.+0.j  6.+0.j]]


In [62]:
print(A2_band_h)

[[0. 1. 1. 1. 1.]
 [2. 2. 2. 2. 2.]]


In [64]:
x1 = linalg.solveh_banded(A1_band_h, b1, lower = False)
x2 = linalg.solveh_banded(A2_band_h, b2, lower = False)

In [66]:
print(x1)

[0.08818236+0.03959208j 0.18116377-0.06778644j 0.17156569+0.04259148j
 0.23095381-0.01439712j]


In [67]:
print(x2)

[0.5 0.  0.5 0.  0.5]




### Why Use Specialized Solvers for Band Matrices?
The primary reason for using specialized solvers for band matrices is computational efficiency. 

#### Time Comparison
To illustrate this, we compare the time taken to solve a 1000 x 1000 tridiagonal system using both the standard full matrix solver (`solve`) and the band matrix solver (`solve_banded`):

- **General Matrix Solver (`solve`)**: ~ **51.08 seconds**
- **Band Matrix Solver (`solve_banded`)**: ~ **0.0003 seconds (0.3 milliseconds)**

The band matrix solver is significantly faster because it only considers the non-zero entries within the band, skipping over the unnecessary zero entries. As matrix sizes increase, the efficiency improvement becomes even more pronounced.



In [70]:
np.random.seed(0)
n = 1000
main_diag = 2 * np.ones(n)
upper_diag = -1 * np.ones(n - 1)
lower_diag = -1 * np.ones(n - 1)

ab = np.zeros((3, n))
ab[0, 1:] = upper_diag
ab[1, :] = main_diag
ab[2, :-1] = lower_diag

A_full = np.diag(main_diag) + np.diag(upper_diag, 1) + np.diag(lower_diag, -1)

b = np.random.randn(n)

start_full = time.time()
x_full = linalg.solve(A_full, b)
time_full = time.time() - start_full

start_banded = time.time()
x_banded = linalg.solve_banded((1, 1), ab, b)
time_banded = time.time() - start_banded

(time_full, time_banded)

(0.022999286651611328, 0.0)